In [1]:
import pandas as pd
import os
from random import shuffle
import numpy as np

In [2]:
def get_time_frame(df, M, D):
    df_M = df[df['M'] == M]
    df_D = df_M[df_M['D'] == D]
#     df_HH = df_D[df_D['HH'] == HH]
#     df_MM = df_HH[df_HH['MM'] == MM]
    return df_D

In [3]:
df_segments = pd.read_csv(r'../../Data2/Dataset_Prepared/Road_Segments_Dir/New_Road_Segments_Divided_2019.csv')
df_segments = df_segments[['SegmentID_', 'TrafDir','SegDir', 'ID']]
df_count_0 = pd.read_csv(r'../../Dataset/Automated_Traffic_Volume_Counts.csv')

In [4]:
SegID = df_count_0['SegmentID'].astype('str')
Dir = df_count_0['Direction']
Seg_ID = SegID+'_' + Dir
df_count_0['Seg_Dir'] = Seg_ID

count_d = df_count_0['Vol']
m = count_d.mean()
s = count_d.std()
print(m, s)

106.66053597731798 156.377030788485


In [5]:
year_list = [2013, 2014, 2015, 2016, 2017, 2018, 2019]

for year in year_list:
    print(year)

    choose_days_per_year = 50

    df_count = df_count_0[df_count_0['Yr'] == year]
    Yr = df_count['Yr'].to_list()
    M = df_count['M'].to_list()
    D = df_count['D'].to_list()
    time = []

    for i in range(len(Yr)):
        t = str(Yr[i]) + '_' + str(M[i]) + '_' + str(D[i])
        time.append(t)

    time_list = list(set(time))
    print('time len is: ' + str(len(time_list)))

    if choose_days_per_year > len(time_list):
        choose_days_per_year = len(time_list)

    shuffle(time_list)
    time_list = time_list[0:choose_days_per_year]


    for time in time_list:
        Day_time_series_data = np.full((df_segments.shape[0], 96), -1.0)
        Day_list = [-1.0] * df_segments.shape[0]
        T = time.split('_')
        M = int(T[1])
        D = int(T[2])
        time_frame = get_time_frame(df_count, M, D)

        seg_dir_list = list(set(time_frame['Seg_Dir'].to_list()))

        for seg_dir in seg_dir_list:
            df_time_seg = time_frame[time_frame['Seg_Dir']==seg_dir]
            if (df_time_seg.shape[0] == 96):
                seg = int(seg_dir.split('_')[0])
                direction = seg_dir.split('_')[1]
                count = (df_time_seg['Vol'] - m)/s
#                 print(count)
                seg_df = df_segments[df_segments['SegmentID_'] == seg]
                if seg_df.shape[0] == 0:
#                     print('No such segment: ' + str(seg))
                    continue
                else:
                    if seg_df['TrafDir'].values[0] != 'T':
                        index = seg_df['ID'].values[0]
                        Day_time_series_data[index] = count
                    else:
                        seg_dir_df = seg_df[seg_df['SegDir']==direction]
                        if seg_dir_df.shape[0] == 0:
#                             print('Have segment, but no direction: ' + str(seg))
                            continue
                        index = seg_dir_df['ID'].values[0]
                        Day_time_series_data[index] = count
        df = pd.DataFrame(Day_time_series_data, columns=[str(c) for c in range(96)])
        name = time + '.csv'
        path = os.path.join(r'../../Data2/Dataset_Prepared/Y_Day_timeseries/Train', name)
        df.to_csv(path, float_format=str)

2013
time len is: 348
2014
time len is: 347
2015
time len is: 331
2016
time len is: 342
2017
time len is: 338
2018
time len is: 283
2019
time len is: 329
